## Pytorch autograd (자동 미분)

In [2]:
import torch

In [3]:
# 역전파를 위해서 X에 대한 gradient 계산 형성
x = torch.FloatTensor([[1, 2],
                        [3, 4]]).requires_grad_(True)

requires_grad_(True) : 이 텐서에 대해서 미분값을 구하겠다.   
일반적인 학습에서는 필요없지만 입력에 대한 기울기 분석시에 사용

In [ ]:
# 연산트리 자동생성 
x1 = x + 2      # 모든 원소에 +2
x2 = x - 2      # 모든 원소에 -2
x3 = x1 * x2    # (x+2) 와 (x-2) 텐서를 원소별로 곱함
y = x3.sum()    # 모든 원소를 더해 스칼라값으로 변환

print(x1)
print(x2)
print(x3)
print(y)

tensor([[3., 4.],
        [5., 6.]], grad_fn=<AddBackward0>)
tensor([[-1.,  0.],
        [ 1.,  2.]], grad_fn=<SubBackward0>)
tensor([[-3.,  0.],
        [ 5., 12.]], grad_fn=<MulBackward0>)
tensor(14., grad_fn=<SumBackward0>)


### grad_fn 이 각 연산을 추적

In [5]:
# 자동 미분을 통한 역전파 
y.backward()    # 스칼라값 y를 기준으로 연결된 모든 텐서의 gradient를 자동으로 계산 

In [6]:
# 역전파 이후 x 에 대한 손실함수의 기울기(미분값) 확인
print(x.grad) # y에 대해 계산된 x의 gradient(∂y/∂x) 출력

tensor([[2., 4.],
        [6., 8.]])


x =    
tensor[[1, 2],   
        [3, 4]]   

=> 편미분값  
tensor([[2., 4.],  
        [6., 8.]])

y = Σ(x+2)(x-2)   
=>
Σ($x^2$ - 4)
각 원소에 대해 $x^2$ - 4를 계산한 후에 전부 더한 것

- 상수를 미분하면?
$\frac{\partial}{\partial x}(-4)$ = 0
=> -4는 미분에 영향을 주지 않는다.
- 제곱항을 주게되면?
$\frac{\partial}{\partial x_i}(x_i^2) = 2x_i$ 

- 각 원소에 대해서 결과는
$\frac{\partial y}{\partial x_i} = 2x_i$

- 이걸 벡터화시키게 되면 
$\frac{\partial y}{\partial x} = 2x$

=> x.grad 출력 시 각 원소에 대해 x2값을 가지게 된다.

In [8]:
print(x) # 텐서값은 변하지않음. 역전파 시 기울기는 계산하지만 값을 수정하지는 않는다


tensor([[1., 2.],
        [3., 4.]], requires_grad=True)


실제 값을 수정할때는    optimizer.step()   
최적화함수 사용하여 가중치 수정한다

In [ ]:
x3.numpy() # RuntimeError # 계산 그래프가 연결된 텐서는 numpy변환을 직접 허용하지 않는다.

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

numpy는 autograd를 모르기때문에 허용하지 않는다.  
즉 미분 추적을 깨트릴 수 있기 때문에(무결성)

In [10]:
x3.detach_().numpy() # detach_ == inplace 로 계산 그래프 분리 

array([[-3.,  0.],
       [ 5., 12.]], dtype=float32)

- detach 와 detach_

| 구분 | detach() | detach_() |
|-----|---------|-----------|
| 동작 방식 | 복사본 만들어 분리 | 원본 자체를 분리 |
| 원본 손상 여부 | 손상 없음 | 원본이 autograd 추적 불가 상태로 바뀐다 |
| 사용 시점 | 일반적으로 사용 | 성능이 매우 중요할 때(추적 불가 - 주의해서 사용) |

In [14]:
x = torch.FloatTensor([[1, 2],
                        [3, 4]]).requires_grad_(True)
x1 = x + 2      # 모든 원소에 +2
x2 = x - 2      # 모든 원소에 -2
x3 = x1 * x2    # (x+2) 와 (x-2) 텐서를 원소별로 곱함

print(x3) # autograd 연결 

tensor([[-3.,  0.],
        [ 5., 12.]], grad_fn=<MulBackward0>)


In [12]:
x3.detach_()

print(x3) # autograd 연결 해제

tensor([[-3.,  0.],
        [ 5., 12.]])


In [13]:
print(x3.numpy)

<built-in method numpy of Tensor object at 0x10f5b5810>
